In [ ]:
import torch
import torchvision
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_V2_Weights, maskrcnn_resnet50_fpn_v2
from PIL import Image
torch.__version__

In [ ]:
weights = MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT
m = maskrcnn_resnet50_fpn_v2(weights=weights)
m = m.eval()

In [ ]:
image = Image.open("cat.jpg")
preprocess = weights.transforms()
data = preprocess(image)
data = data.unsqueeze(0)
print(data.shape)

In [ ]:
res = m(data)
print(res)

In [6]:
import time
import numpy as np
from tqdm import tqdm

import torch._dynamo
torch._dynamo.reset()

#mode = None # 58, 55
#mode = "default" # 56, 55
#mode = "max-autotune" # 57, 55
mode = "reduce-overhead" # 58, 55
print("Compiling...")
m_opt = torch.compile(m, mode=mode)
res = m_opt(data)
print("Compilation Done")

for model, desc in [(m, "maskrcnn"), (m_opt, "maskrcnn_inductor_reduce-overhead")]:
    N = 3; i = 0
    while i < N:
        res = model(data)
        i += 1

    N = 10; i = 0
    TT = []
    while i < N:
        t0 = time.time()
        res = model(data)
        dur = (time.time() - t0) * 1000
        TT.append(dur)
        i += 1

    print(f"{desc},{np.mean(TT):.3f},{np.percentile(TT, 50):.3f}")

Compiling...
Compilation Done
def: mean time: 2260.721 ms
def: p50 time: 2283.098 ms
opt: mean time: 2219.373 ms
opt: p50 time: 2173.701 ms


In [ ]:
import time
import numpy as np
from tqdm import tqdm

import torch._dynamo
torch._dynamo.reset()

def evaluate(mod, inp):
    return mod(inp)

print("Compiling...")
#mode = None # 58, 55
#mode = "default" # 56, 55
#mode = "max-autotune" # 57, 55
model = "reduce-overhead" # 58, 55
evaluate_opt = torch.compile(evaluate, mode=mode)
res = evaluate_opt(m, data)
print("Compilation Done")

for fun, desc in [(evaluate, "def"), (evaluate_opt, "opt")]:
    N = 10; i = 0
    while i < N:
        res = fun(m, data)
        i += 1

    N = 50; i = 0
    TT = []
    while i < N:
        t0 = time.time()
        res = fun(m, data)
        dur = (time.time() - t0) * 1000
        TT.append(dur)
        i += 1

    print(f"{desc}: mean time: {np.mean(TT):.3f} ms")
    print(f"{desc}: p50 time: {np.percentile(TT, 50):.3f} ms")

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

aa={
'PyTorch': 92772,
'Tensorflow':7210,
'JAX': 8348,
'ONNX': 4138,
'Keras': 802,
'MXNET': 0
}
people = aa.keys()
y_pos = np.arange(len(people))
performance = aa.values()
error = np.random.rand(len(people))

fig, ax = plt.subplots()

hbars = ax.barh(y_pos, performance, xerr=error, align='center')
ax.set_yticks(y_pos, labels=people)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Model numbers')
ax.set_title('Hugging Face Stat')

# Label with specially formatted floats
ax.bar_label(hbars, fmt='{:,.0f}')
ax.set_xlim(right=105000)  # adjust xlim to fit labels

plt.show()